## Import's

In [1]:
import tensorflow as tf
from datasets import Dataset, concatenate_datasets
from transformers import AutoTokenizer
from transformers import TFAutoModelForSequenceClassification
from transformers import create_optimizer
from keras import backend as K
import numpy as np
import json
import os

## Verificando Tensoflow-GPU

In [2]:
lista_gpu = tf.config.list_physical_devices("GPU")
"Tem GPU" if lista_gpu else "Não tem GPU"

2023-01-19 11:15:28.801377: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


'Tem GPU'

2023-01-19 11:15:28.807978: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-19 11:15:28.808136: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-19 11:15:28.808188: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1953] TensorFlow was not built with CUDA kernel binaries compatible with compute capability 8.6. CUDA kernels will be jit-compiled from PTX, which could take 30 minutes or longer.


In [3]:
for gpu in lista_gpu:
    tf.config.experimental.set_memory_growth(gpu, True)

## Constantes

In [4]:
## Constantes de diretório
DIR = '../' ## Diretório 
dataset_path = "data/" ## Diretório raiz dos dataset's
dir_resultado = "results/" ## Diretório raiz para os resultados

## Constantes do modelo
model_id = 'neuralmind/bert-base-portuguese-cased'
epochs = 3
batchs = 16
qtd_classes = 3
tamanho_cross_validation = 10

num_train_steps = (tamanho_cross_validation  // batchs) * epochs

## Tokenizador

In [5]:
tokenizer = AutoTokenizer.from_pretrained(model_id)

In [6]:
def tokenize_dataset(data):
    return tokenizer(data["text"], padding=True, return_tensors="tf")

## Modelo

In [7]:
model = TFAutoModelForSequenceClassification.from_pretrained(model_id, num_labels=qtd_classes)

Downloading:   0%|          | 0.00/529M [00:00<?, ?B/s]

2023-01-19 11:21:35.006860: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-19 11:21:35.007351: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-19 11:21:35.007500: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-19 11:21:35.007612: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so retur

## Lendo datasets

In [8]:
dict_datasets : dict[str, Dataset] = dict({})
for arquivo_nome in list(filter(lambda x: x.endswith(".json"), os.listdir(DIR+dataset_path))):
    dict_datasets.update(dict({arquivo_nome: Dataset.from_json(DIR+dataset_path+arquivo_nome)}))

Using custom data configuration default-4236bfd91f2c18b5
Found cached dataset json (/home/thiago/.cache/huggingface/datasets/json/default-4236bfd91f2c18b5/0.0.0)
Using custom data configuration default-6cb4c78ed98faec2
Found cached dataset json (/home/thiago/.cache/huggingface/datasets/json/default-6cb4c78ed98faec2/0.0.0)
Using custom data configuration default-75cea81a1a2a16ff
Found cached dataset json (/home/thiago/.cache/huggingface/datasets/json/default-75cea81a1a2a16ff/0.0.0)
Using custom data configuration default-c79f6086a25a6b5a
Found cached dataset json (/home/thiago/.cache/huggingface/datasets/json/default-c79f6086a25a6b5a/0.0.0)
Using custom data configuration default-eb7ec91bbd4f537b
Found cached dataset json (/home/thiago/.cache/huggingface/datasets/json/default-eb7ec91bbd4f537b/0.0.0)
Using custom data configuration default-403847b7b10ac345
Found cached dataset json (/home/thiago/.cache/huggingface/datasets/json/default-403847b7b10ac345/0.0.0)
Using custom data configurat

In [9]:
dict_datasets

{'dataset_olist.json': Dataset({
     features: ['text', 'labels', 'group'],
     num_rows: 38079
 }),
 'dataset_utlc_apps.json': Dataset({
     features: ['text', 'labels', 'group'],
     num_rows: 968858
 }),
 'dataset_concatenate.json': Dataset({
     features: ['text', 'labels', 'group'],
     num_rows: 2400951
 }),
 'dataset_buscape.json': Dataset({
     features: ['text', 'labels', 'group'],
     num_rows: 73626
 }),
 'dataset_utlc_movies.json': Dataset({
     features: ['text', 'labels', 'group'],
     num_rows: 1189542
 }),
 'dataset_b2w.json': Dataset({
     features: ['text', 'labels', 'group'],
     num_rows: 116058
 }),
 'dataset_twitter_ASBR.json': Dataset({
     features: ['labels', 'text', 'group'],
     num_rows: 14788
 })}

## Tokenizando as entradas

In [ ]:
for dataset_nome, dataset in dict_datasets.items():
    dict_datasets[dataset_nome] = dataset.map(tokenize_dataset, batched=True)

In [ ]:
dict_datasets

## Dividindo os dados

In [ ]:
def group_by(dataset: Dataset):
    lista_grupos = []
    for grupo in set(dataset["group"]):
        lista_grupos.append(dataset.filter(lambda x: x["group"]==grupo))
    return lista_grupos

In [ ]:
for dataset_nome, dataset in dict_datasets.items():
    dict_datasets[dataset_nome] = group_by(dataset)

In [ ]:
dict_datasets

## Metricas

In [ ]:
def func_precision(y_true, y_pred):
    y_pred = np.argmax(y_pred.numpy(), axis=1)
    y_true = np.argmax(y_true.numpy(), axis=1)
    precision = tf.keras.metrics.Precision()
    precision.update_state(y_true, y_pred)
    return precision.result().numpy()

def func_recall(y_true, y_pred):
    y_pred = np.argmax(y_pred.numpy(), axis=1)
    y_true = np.argmax(y_true.numpy(), axis=1)
    recall = tf.keras.metrics.Recall()
    recall.update_state(y_true, y_pred)
    return recall.result().numpy()

def func_f1(y_true, y_pred):
    precision = func_precision(y_true, y_pred)
    recall = func_recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

## Criando otimizador

In [ ]:
optimizer, lr_schedule = create_optimizer(
    init_lr=2e-5,
    num_train_steps=num_train_steps,
    weight_decay_rate=0.01,
    num_warmup_steps=0,
)

## Treinando o modelo

In [ ]:


for nome_dataset, list_dataset in dict_datasets.items():
    resultados = []
    for index, dataset in enumerate(list_dataset):
        ## Treino
        dataset_treino = list_dataset[:index]
        dataset_treino += list_dataset[index+1:]
        dataset_treino = concatenate_datasets(dataset_treino)
        tf_dataset_treino = model.prepare_tf_dataset(dataset_treino, batch_size=batchs, shuffle=True, tokenizer=tokenizer)
        ## Teste
        tf_dataset_teste = model.prepare_tf_dataset(dataset, batch_size=batchs, shuffle=True, tokenizer=tokenizer)
        ## Modelo
        model.compile(
            optimizer=optimizer,
            loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
            metrics=[
                tf.keras.metrics.CategoricalAccuracy(),
                func_precision,
                func_recall,
                func_f1
            ],
            run_eagerly = True
        )
        history = model.fit(tf_dataset_treino, epochs=epochs, use_multiprocessing=True)
        loss, acc, precision, recall, f1 = model.evaluate(tf_dataset_teste, use_multiprocessing=True)
        resultados.append(dict({
            "loss" : loss,
            "accuracy" : acc,
            "precision" : precision,
            "recall" : recall,
            "f1" : f1
        }))
    with open(f"{dir}{dir_resultado}{dataset_nome}.json", "w") as arquivo:
        json.dump(resultados, arquivo, indent=4)